----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery layer - OMM Stream - Level 1 data subscription
This notebook demonstrates how to use an OMM stream of the low level Delivery layer to subscribe to retrieve level 1 streaming data (such as trades and quotes).

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [1]:
import refinitiv.data as rd
from refinitiv.data.delivery import omm_stream
import datetime
import json

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7faa8e2242b0 {name='workspace'}>

## Retrieve data

### Define a calback to display received events

In [3]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(json.dumps(event, indent=2))
    return

### Create an OMM stream and register event callbacks

In [4]:
stream = omm_stream.Definition(
    name="EUR=", 
    fields=['BID', 'ASK','OPEN_PRC','HST_CLOSE','TIMACT'], 
    domain='MarketPrice').get_stream()

# Refresh - the first full image we get back from the server
stream.on_refresh(lambda event, item_stream  : display_event("Refresh", event))

# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda event, item_stream : display_event("Update", event))

# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda event, item_stream : display_event("Status", event))

# Other errors
stream.on_error(lambda event, item_stream : display_event("Error", event))

### Open the stream
This example uses the open() synchronous call to open the stream. Because it is a synchronous call, the first notification (either via on_refresh(), on_status() or on_error()) happens before the open() method returns. If the open_async() asynchronous method is used instead, the first notification happens after open_async() returns.

In [5]:
# Send request to server and open stream
stream.open()
# We should receive the initial Refresh for the current field values
# followed by updates for the fields as and when they occur

----------------------------------------------------------
>>> Refresh event received at 15:54:48.630131
{
  "ID": 5,
  "Type": "Refresh",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "New Session Mounted"
  },
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "PermData": "AwD+Umw=",
  "SeqNumber": 4478,
  "Fields": {
    "BID": 1.0566,
    "ASK": 1.057,
    "OPEN_PRC": 1.0525,
    "HST_CLOSE": 1.0525,
    "TIMACT": "13:54:00"
  }
}


<OpenState.Opened: 'Opened'>

----------------------------------------------------------
>>> Update event received at 15:54:48.796412
{
  "ID": 5,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "SeqNumber": 4542,
  "Fields": {
    "BID": 1.0567,
    "ASK": 1.0568,
    "TIMACT": "13:54:00"
  }
}
----------------------------------------------------------
>>> Update event received at 15:54:49.230649
{
  "ID": 5,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "SeqNumber": 4558,
  "Fields": {
    "BID": 1.0565,
    "ASK": 1.0569,
    "TIMACT": "13:54:00"
  }
}
----------------------------------------------------------
>>> Update event received at 15:54:50.077639
{
  "ID": 5,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "SeqNumber": 4622,
  "Fields": {
    "BID": 1.0567,
    "ASK": 1.0569,
    "TIMACT": 

### Close the stream

In [6]:
stream.close()

----------------------------------------------------------
>>> Update event received at 15:54:55.271527
{
  "ID": 5,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "SeqNumber": 5438,
  "Fields": {
    "BID": 1.0568,
    "ASK": 1.0569,
    "TIMACT": "13:54:00"
  }
}


<OpenState.Closed: 'Closed'>

## Close the session

In [7]:
rd.close_session()